# Canary Roll Out of ML Models with Seldon and Istio

This folder provides resources to illustrate how to do a canary	roll out of one	MNIST model to another using the canary	pattern where a small amount	of traffic is sent to the new model to validate	it before sending all traffic to the new model.

We utilize two MNIST digit classification models. 

 * Version 1 of the model using R
 * Version 2 of the model using Tensorflow.
 
After deploying Istio and Seldon to a kubernetes cluster we will:

 * Deploy version 1 R based model using seldon-core
 * Create an istio routing rule to direct all traffic to this version
 * Create a canary deployment with both version 1 and version 2 (the Tensorflow model)
 * Update the istio routing to send 10% of the traffic to version 2
 * Update the istio routing to send 100% of the traffic to version 2
 
 

# Setup

The steps below will install istio and seldon onto a GKE Cluster. If you wish to use your own setup then you need to ensure

 * You allow istio egress to the internet as the load test downloads MNIST images
 * Ensure you give your user cluster-admin privledges
 * Install seldon into a namespace seldon
 
To follow the steps below you will need:
 
  * a Google project running a K8S cluster
  * gcloud and kubectl installed
  * istio download

## Setup Environment Variables

In [ ]:
%env ISTIO_HOME=/home/clive/work/istio/istio-0.8.0

In [ ]:
%env ZONE=europe-west1-d

In [ ]:
%env PROJECT=my-project

## Determine CIDR ranges

In [ ]:
gcloud container clusters describe cluster-istio-1 --zone ${ZONE} --project ${PROJECT} | grep -e clusterIpv4Cidr -e servicesIpv4Cidr

## Install Helm

In [ ]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

In [ ]:
!kubectl create -f ${ISTIO_HOME}/install/kubernetes/helm/helm-service-account.yaml

In [ ]:
!helm init --service-account tiller

## Install Istio

** Replace the CIDR values with those you got above **

For more details see [istio docs on egress](https://istio.io/docs/tasks/traffic-management/egress/#calling-external-services-directly)

In [ ]:
!helm install ${ISTIO_HOME}/install/kubernetes/helm/istio --name istio --namespace istio-system \
    --set global.proxy.includeIPRanges="10.20.0.0/14\,10.23.240.0/20"

In [ ]:
!kubectl apply -f ${ISTIO_HOME}/install/kubernetes/addons/grafana.yaml

To view the istio Grafana dashboard:
```
kubectl -n istio-system port-forward $(kubectl -n istio-system get pod -l app=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000
```
http://localhost:3000/dashboard/db/istio-dashboard

## Install Seldon

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl apply -f ../../../notebooks/resources/ambassador-rbac.yaml -n seldon

To send requests to Ambassador ingress:
    
```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambaador -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:80
```

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd

In [ ]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon

In [ ]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

To view the Seldon Grafana dashboard:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3001:3000
```

http://localhost:3001/dashboard/db/prediction-analytics?refresh=5s&orgId=1

In [ ]:
!kubectl label namespace seldon istio-injection=enabled

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

# Launch Version 1 Model

In [ ]:
%matplotlib inline
import utils
mnist = utils.download_mnist()

In [ ]:
!pygmentize mnist_v1.json

In [ ]:
!kubectl create -f mnist_v1.json

** Wait until new pods are running **

In [ ]:
!kubectl get pods

In [ ]:
utils.predict_rest_mnist(mnist,"mnist-classifier")

## Start a Load Test

In [ ]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust

In [ ]:
!helm install seldon-core-loadtesting --name loadtest  \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts \
    --set locust.script=mnist_rest_locust.py \
    --set locust.host=http://mnist-deployment:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=1 \
    --set loadtest.sendFeedback=1 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=1 \
    --set data.size=784


# Setup Version 1 Istio Routing

In [ ]:
!pygmentize istio_canary_v1.yaml

In [ ]:
!istioctl create -f istio_canary_v1.yaml

# Launch Version 2 of Model

In [ ]:
!pygmentize mnist_v2.json

In [ ]:
!kubectl apply -f mnist_v2.json

** Wait until new pods are running **

In [ ]:
!kubectl get pods

In [ ]:
utils.predict_rest_mnist(mnist,"mnist-classifier")

# Setup Canary Routing

In [ ]:
!pygmentize istio_canary_v2.yaml

In [ ]:
!istioctl replace -f istio_canary_v2.yaml

# Setup Routing to Version 2

In [ ]:
!pygmentize istio_canary_v3.yaml

In [ ]:
!istioctl replace -f istio_canary_v3.yaml